In [1]:
%pip install -q kingdon anywidget==0.9.13

Note: you may need to restart the kernel to use updated packages.


# Geometric Gauges: 
# Plane and Simple

**<p style="text-align: right;">Martin Roelfs</p>**
<p style="text-align: right;">University of Antwerp</p>

In [38]:
from kingdon import Algebra, MultiVector
import numpy as np
import itertools
import timeit
import ipywidgets as ipy
from collections.abc import Callable

alg3d = Algebra(3, 0, 1)

options = dict(
    lineWidth=4,
    pointRadius=2.5,
    fontSize=3,
)
animated_options = dict(animate=1, **options)
clrs = [0xff9900, 0xfed290, 0x009977, 0x000000]

In [39]:
# https://enkimute.github.io/ganja.js/examples/coffeeshop.html#XOWMGemyJ

# Initiate a shape
np.random.seed(42)
coords = np.ones((3, 5))
coords[1:3] = np.random.uniform(0.5, 1.5, size=(2, 5))
points1 = alg3d.vector(coords, keys=('e0', 'e1', 'e2')).dual()
points2 = points1.map(lambda v: np.roll(v, 1, axis=-1))
shape = list(zip(points1, points2))

# Reflections
PLANE = alg3d.vector(e3=1)
origin = alg3d.blades.e0.dual() | PLANE
p1 = alg3d.vector(e0=1, e1=0, e2=0).dual()
p2 = alg3d.vector(e0=1, e1=0, e2=1).dual()
# p3 = alg.vector(e0=1, e1=1, e2=0.3).dual()
u = lambda: ((p1 & p2).normalized())
L1 = alg3d.vector(e1=1).normalized() ^ PLANE
L2 = alg3d.vector(e1=1, e2=0.5).normalized() ^ PLANE
L3 = alg3d.vector(e1=1, e0=0.5).normalized() ^ PLANE

pa = alg3d.vector(e0=1, e1=0.5, e2=0).dual()
pb = alg3d.vector(e0=1, e1=-1.5, e2=0).dual()
L4 = alg3d.vector(e2=1).normalized()

line_point_end = alg3d.vector(e0=1, e1=2, e2=0).dual().normalized()
line_point_begin = alg3d.vector(e0=2, e1=2, e2=0).dual().normalized()
line_segment = [line_point_begin, line_point_end]

In [40]:
# Input for the graph showing point-reflections in 1D, tilting into 2D.
line = (L4 ^ PLANE).normalized()
T1 = alg3d.evenmv(e=1.0, e01=0.4)
T2 = alg3d.evenmv(e=1.0, e03=-1.0)
_begin_point_line = alg3d.evenmv(e=1.0, e01=-5) >> pa
_end_point_line = alg3d.evenmv(e=1.0, e01=5) >> pa
vanishing_point = ((line | pa) ^ L4) ^ alg3d.blades.e0

grid_lines = [
    [T1.reverse() >> (T2 >> pa), vanishing_point], 
    [T2.reverse() >> _begin_point_line, T2.reverse() >> _end_point_line]
]
for i in range(5):
    gl1 = grid_lines[-2]
    gl2 = grid_lines[-1]
    grid_lines.append([T1 >> gl1[0], gl1[1]])
    grid_lines.append([~T2 >> gl2[0], ~T2 >> gl2[1]])

t0 = None
def refl_1d_graph_func():
    line_segment_a = [pa >> point for point in line_segment]
    line_segment_b = [pb >> point for point in line_segment_a]
    global t0
    
    if graph.fragment_widget.fragment == -1:
        return [
            [_begin_point_line, _end_point_line],
            pa, 'a',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment,
            line_segment[-1],
            '</G>',
        ]
    elif graph.fragment_widget.fragment == 0:
        return [
            [_begin_point_line, _end_point_line],
            pa, 'a',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment,
            line_segment[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[2],
            line_segment_a,
            line_segment_a[-1],
            '</G>',
        ]
    elif graph.fragment_widget.fragment == 1:
        return [
            [_begin_point_line, _end_point_line],
            pa, 'a',
            pb, 'b',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment,
            line_segment[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[2],
            line_segment_a,
            line_segment_a[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment_b,
            line_segment_b[-1],
            '</G>',
        ]
    elif graph.fragment_widget.fragment >= 2:
        if t0 is None:
            t0 = timeit.default_timer()

        omega = -min((timeit.default_timer() - t0) / 25, np.pi / 20)
        R = np.cos(omega) + line * np.sin(omega)
        
        return [
            [_begin_point_line, _end_point_line],
            '<G stroke-width="0.02" stroke-opacity="0.5">',
            *[[R >> gl[0], R >> gl[1]] for gl in grid_lines],
            '</G>',
            '<G stroke-width="0.05">',
            pa, 'a',
            pb, 'b',
            '</G>',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment,
            line_segment[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[2],
            line_segment_a,
            line_segment_a[-1],
            '</G>',
            '<G stroke-width="0.05">',
            clrs[0],
            line_segment_b,
            line_segment_b[-1],
            '</G>',
        ]

In [41]:
# Graph showing a single reflection in 2D.

def refl_graph_func():
    t = timeit.default_timer() / 50
    # Create the reflected shape and the lines between them
    R = np.cos(t) + origin*np.sin(t)
    L1p = R >> L1
    _rpoints1 = L1p >> points1
    _rpoints2 = L1p >> points2
    rshape = zip(_rpoints1, _rpoints2)
    rlines = zip(_rpoints1, points1)
    
    return [
        L1p,
        clrs[0],
        *shape, 
        clrs[2],
        *rshape,
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        '</G>',
    ]

In [42]:
# Interactive bireflections.

def bi_refl_graph_func():
    # Reflect the points once and then once more.
    rpoints1 = u >> points1
    rrpoints1 = L1 >> rpoints1
    rpoints2 = u >> points2
    rrpoints2 = L1 >> rpoints2
    
    # Create the reflected shape and the lines between them
    rshape = zip(rpoints1, rpoints2)
    rrshape = zip(rrpoints1, rrpoints2)
    rlines = zip(rpoints1, points1)
    rrlines = zip(rpoints1, rrpoints1)
    
    return [
        p1, p2,
        u,
        L1,
        clrs[0],
        *shape, 
        '<G stroke-dasharray="0.02 0.02">',clrs[1],
        *rshape,
        '</G>',
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        *rrlines,
        '</G>',
        clrs[2], 
        *rrshape,
    ]

def point_graph_func():
    # Create the reflected shape and the lines between them
    point_reflected_points1 = p1 >> points1
    point_reflected_points2 = p1 >> points2
    rshape = zip(point_reflected_points1, point_reflected_points2)
    rlines = zip(point_reflected_points1, points1)
    
    return [
        p1,
        clrs[0],
        *shape, 
        clrs[2],
        *rshape,
        '<G stroke-width="0.002">',clrs[0],
        *rlines,
        '</G>',
    ]

In [43]:
# Gauge degree in bireflections animation. # TODO: Show both rotation and translation.
def _birefl_gauge_func(L1, L2, R0=None):
    # R0 is the initial state.
    # if R0:
    #     L1 = R0 >> L1
    #     L2 = R0 >> L2
    #     _points1 = R0 >> points1
    #     _points2 = R0 >> points2
    #     _shape = list(zip(_points1, _points2))
        
    intersection = L1.cp(L2)
    if not intersection**2:
        R = lambda t: 1 + intersection*np.sin(5*t)
    else:
        R = lambda t: np.cos(t) + intersection*np.sin(t)
        
    def _graph_func():
        t = timeit.default_timer() / 30
        L1p = R(t) >> L1
        L2p = R(t) >> L2
        # Reflect the points once and then once more.
        rpoints1 = L1p >> points1
        rrpoints1 = L2p >> rpoints1
        rpoints2 = L1p >> points2
        rrpoints2 = L2p >> rpoints2
        # Create the reflected shape and the lines between them
        rshape = zip(rpoints1, rpoints2)
        rrshape = zip(rrpoints1, rrpoints2)
        rlines = zip(rpoints1, points1)
        rrlines = zip(rpoints1, rrpoints1)
        
        return [
            '<G stroke-opacity="0.4">', clrs[-1],
            L1p,
            L2p,
            '</G>',
            0xff9900,
            *shape, 
            '<G stroke-dasharray="0.02 0.02">',0xfed290,
            *rshape,
            '</G>',
            '<G stroke-width="0.002">',0xff9900,
            *rlines,
            *rrlines,
            '</G>',
            0x009977, 
            *rrshape,
        ]
    return _graph_func

Tleft = alg3d.evenmv(e=1.0, e01=-1.5)
Tright = alg3d.evenmv(e=1.0, e01=1.5)

_birefl_gauge_rot_func = _birefl_gauge_func(L1, L2, R0=Tleft)
_birefl_gauge_trans_func = _birefl_gauge_func(L1, L3, R0=Tright)

In [44]:
# Product of two points, with sliders. # TODO: also show the 3D perspective?
p1plane1 = alg3d.vector(e0=1.0, e1=1.0, e2=0.0).normalized()
p1plane2 = alg3d.vector(e0=1.0, e1=0.0, e2=1.0).normalized()
p2plane1 = alg3d.vector(e0=-1.0, e1=1.0, e2=0.0).normalized()
p2plane2 = alg3d.vector(e0=-1.0, e1=0.0, e2=1.0).normalized()
p1l1 = p1plane1 ^ PLANE
p1l2 = p1plane2 ^ PLANE
axis1 = p1plane1 ^ p1plane2
p1 = axis1 ^ PLANE
p2l1 = p2plane1 ^ PLANE
p2l2 = p2plane2 ^ PLANE
axis2 = p2plane1 ^ p2plane2
p2 = axis2 ^ PLANE

p1_slider = ipy.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.05,
    description='',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
p2_slider = ipy.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.05,
    description='',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

def points_gp_graph_func():
    R1 = np.cos(0.25*np.pi*p1_slider.value) + axis1*np.sin(0.25*np.pi*p1_slider.value)
    R2 = np.cos(0.25*np.pi*p2_slider.value) + axis2*np.sin(0.25*np.pi*p2_slider.value)
    _p1l1 = R1 >> p1l1
    _p1l2 = R1 >> p1l2
    _p2l1 = R2 >> p2l1
    _p2l2 = R2 >> p2l2
    _p1plane1 = R1 >> p1plane1
    _p2plane1 = R2 >> p2plane1
    
    # _p1 = _p1l1.cp(_p1l2) ^ PLANE
    # _p2 = _p2l1.cp(_p2l2) ^ PLANE
    _p1 = p1
    _p2 = p2
    if abs((_p1plane1 ^ _p2).dual().e) < 1e-3 and abs((_p2plane1 ^ _p1).dual().e) < 1e-3:
        return [
            clrs[0],
            _p1l2, _p1, 'a',
            '<G stroke-opacity="0.1">',
            _p1l1,
            '</G>',
            clrs[2],
            _p2l2, _p2, "b",
            '<G stroke-dasharray="0.1 0.1" stroke-opacity="0.1">',
            _p2l1,
            '</G>',
        ]
    return [
        clrs[0],
        _p1l1, _p1l2, _p1, 'a',
        clrs[2],
        _p2l1, _p2l2, _p2, "b",
    ]


In [45]:
sw_slider = ipy.FloatSlider(
    value=0.0,
    min=-1.0,
    max=1.0,
    step=0.05,
    description='',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

def sw_graph_func():
    if sw_slider.value == 0.0:
        return [
            clrs[0],
            L1, 'a',
            clrs[2],
            L2, "b'",
            '<G stroke-dasharray="0.1 0.1">', clrs[1],
            L1, "a'",
            '</G>',
        ]
    elif sw_slider.value < 1.0:
        t = np.arccos((L1 | L2).e) * sw_slider.value
        intersection = L1 ^ L2
        R = np.cos(t) + intersection*np.sin(t)
        L2p = R >> L2
        L1p = R >> L1
        
        return [
            clrs[0],
            L1, 'a',
            clrs[2],
            L2p, "b'",
            clrs[1],
            L1p, "a'",
        ]
    else:
        R = (L1 * L2).sqrt()
        L2p = R >> L2
        L1p = R >> L1
        
        return [
            '<G fill-opacity="0.3" stroke-opacity="0.1">', 
            clrs[0],
            L1, 'a',
            '</G>',
            '<G stroke-opacity="0.3" fill-opacity="0.3" stroke-dasharray="0.1 0.1">', 
            clrs[2],
            L2p, "b'",
            '</G>',
            clrs[0],
            L1p, "a'",
        ]

In [55]:
import anywidget
import traitlets

from reveal_widgets import FragmentWidget
from kingdon.graph import GraphWidget

graph_funcs = {
    (0, 0): lambda: [],
    (1, 0): refl_1d_graph_func,
    (1, 1): refl_graph_func,
    (1, 2): bi_refl_graph_func,
    (1, 3): _birefl_gauge_trans_func,
    (1, 4): points_gp_graph_func,
    (1, 5): sw_graph_func,
}

class RevealWidget(GraphWidget):
    fragment_widget = traitlets.Instance(FragmentWidget, args=tuple(), kwargs=dict())
    graph_funcs = traitlets.Dict({})

    def __init__(self, *args, raw_subjects, **kwargs):
        print(raw_subjects)
        kwargs['graph_funcs'] = raw_subjects[0]
        super().__init__(*args, raw_subjects=(raw_subjects[0][(0, 0)],), **kwargs)
        self.set_notifiers()
        
    def set_notifiers(self):
        self.fragment_widget.observe(self.changed_state, ['state', 'fragment'])
        
    def changed_state(self, change):
        # print(change)
        fw = change['owner']
        slide_state = fw.state
        if slide_state in self.graph_funcs:
            self.raw_subjects = [self.graph_funcs[slide_state]]
            self.pre_subjects = self.get_pre_subjects()
            # self.subjects = self.get_subjects()
            self.draggable_points_idxs = self.get_draggable_points_idxs()
        # self.draggable_points = self.get_draggable_points()
        

# fragment_widget = FragmentWidget()
# fragment_widget.slide = 1
# fragment_widget.subslide = 0
# fragment_widget.fragment = 1
# fragment_widget

In [56]:
# def nested_transform(subject, R):
#     if isinstance(subject, (MultiVector, Callable)):
#         return R >> subject
#     elif isinstance(subject, (list, tuple)):
#         return [nested_transform(s, R) for s in subject]
#     return subject

# def graph_func():
#     """ Returns the right graph function on the basis of the currenty slide. """
#     if fragment_widget.slide_state == (1, 0):
#         return refl_1d_graph_func()
#     elif fragment_widget.slide_state == (1, 1):
#         return refl_graph_func()
#     elif fragment_widget.slide_state == (1, 2):
#         if fragment_widget.fragment >= 1:
#             return point_graph_func()
#         return bi_refl_graph_func()
#     elif fragment_widget.slide_state == (1, 3):
#         # left = nested_transform(_birefl_gauge_rot_func(), Tleft)
#         # right = nested_transform(_birefl_gauge_trans_func(), Tright)
#         left = _birefl_gauge_rot_func()
#         right = []#_birefl_gauge_trans_func()
#         return [*left, *right]
#     elif fragment_widget.slide_state == (1, 4):
#         return points_gp_graph_func()
#     elif fragment_widget.slide_state == (1, 5):
#         return sw_graph_func()
#     return [origin]

# fragment_widget.observe(graph_func_selector, names=["slide", "subslide", "fragment"])

graph = alg3d.graph(
    graph_funcs, **animated_options, graph_widget=RevealWidget
)

slide_slider = ipy.IntText(
    value=graph.fragment_widget.slide,
    description='Slide:',
    disabled=False
)
subslide_slider = ipy.IntText(
    value=graph.fragment_widget.subslide,
    description='Subslide:',
    disabled=False
)
fragment_slider = ipy.IntText(
    value=graph.fragment_widget.fragment,
    description='Fragment:',
    disabled=False
)
_ = ipy.link((slide_slider, 'value'), (graph.fragment_widget, 'slide'))
_ = ipy.link((subslide_slider, 'value'), (graph.fragment_widget, 'subslide'))
_ = ipy.link((fragment_slider, 'value'), (graph.fragment_widget, 'fragment'))

({(0, 0): <function <lambda> at 0x11cf6a0e0>, (1, 0): <function refl_1d_graph_func at 0x11cf68e50>, (1, 1): <function refl_graph_func at 0x109a58c10>, (1, 2): <function bi_refl_graph_func at 0x11cf69900>, (1, 3): <function _birefl_gauge_func.<locals>._graph_func at 0x11cf3dab0>, (1, 4): <function points_gp_graph_func at 0x11cf689d0>, (1, 5): <function sw_graph_func at 0x11cf688b0>},)


In [61]:
graph.fragment_widget.state, graph.raw_subjects

((1, 2), [<function __main__.bi_refl_graph_func()>])

In [58]:
ipy.VBox([slide_slider, subslide_slider, fragment_slider, graph])

## Introduction

- About Reflections

## About Reflections

<span class="fragment">In $1$ dimension, a single point-reflection inverts all points on the line.<span class="fragment">
<span class="fragment">Two point-reflections on the line make a translation along the line.</span>
<p class="fragment" style="text-align: center;">But how do you know this scenario was 1d?</p>

<p class="fragment" style="text-align: center;">
As we go to higher dimensions, these properties should still hold: 
- one point-reflections inverts any line through that point.
- two point-reflections to create a translation along the line.
</p>

In [ ]:
graph

In $d$ dimensions, a single reflection inverts the entire space except for the $d-1$ dimensional subspace in which we reflect.

In [ ]:
graph

## Bireflections

Two reflections can form a translation, a rotation, or a boost. These are all examples of **bireflections**.

In [ ]:
graph

- Identity is the special case of two identical reflections $\implies$ Reflections are *involutary*.
  > Doing the same reflection twice is the same as doing nothing

- Two orthogonal reflections are another special case, which is identical to a point reflection.
  > A point(-reflection) **is** two orthogonal line(-reflection)s.

<p style="text-align: center;">All bireflections are <b>boring</b> plus <b>drastic</b>.</p>

## Bireflections have Geometric Gauges

Given the input and output shape, you could reconstruct the bireflection between them. However, *you could never know which specific reflections were used to make that bireflection!* 
**<p style="text-align: center;">Bireflections have a gauge degree of freedom!</p>**

In [ ]:
graph

## Product between two points

The product between two points $a$ and $b$: $R = ab$.

In [ ]:
ipy.VBox([p1_slider, p2_slider, graph])

The product of two points is a translation along the line $a \vee b$! In fact, it is **twice** the translation from $a$ to $b$.

## Cartan-Dieudonné theorem

**Geometric Gauges** are the geometric mechanism behind the famous *Cartan-Dieudonné theorem*:
> Every orthogonal transformation in an $n$-dimensional embedding space is composed from at most $n$ reflections. 

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#Mqs0ezNVP&fullscreen" width="100%" height="100%" frameBorder="0" scrolling="no"></iframe>

<p style="text-align: center;">In fact: <b>Geometric Gauges explain all Products!</b></p>

## Inventing Conjugation

How do we reflect the line $b$ over the $a$? Gauges show that we need the product $aba$.
- Use the gauge degree of freedom in $ab = a'b'$ until $b' = a$, at which point $ a b a = (a' b') a  = a' ( b' a) = a'$

In [ ]:
ipy.VBox([sw_slider, graph])

<p style="text-align: center;">Gauges explain the conjugation formula $aba^{-1}$ for $a,b \in \text{Pin}(p, q, r)$!</p>

## Algebra of Geometry

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#8lN3hOoNj&fullscreen" width="100%" height="100%" frameBorder="0" alt="iframe"></iframe>

# Invariant Decomposition

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#ffeKS-07k&fullscreen" width="100%" height="100%" frameBorder="0" alt="iframe"></iframe>

fragement1

fragement2

fragement3

## First Principle

## Invariant Decomposition

$$ b_i = \frac{\lambda_i B + \tfrac{1}{3!} B \wedge B \wedge B}{\lambda_i + \tfrac{1}{2} B \wedge B} $$

## Conclusion

Live demo? This entire presentation has been a live demo!
End by qouting Charles like in the GSG paper?


Scan this QR code to try the kingdon teahouse:

<iframe src="https://enkimute.github.io/ganja.js/examples/coffeeshop.html#XOWMGemyJ&fullscreen" width="100%" height="100%" frameBorder="0" alt="iframe"></iframe>